In [20]:
import pandas as pd # import pandas for file manipulation, read and write, plus other functions
import yfinance as yf # import yfinance to pull stock data from the market
import os # import os to check if directory and file exist
from openpyxl import Workbook # import Workbook to create files
import time

pd.options.display.float_format = '{:.2f}'.format # display float numbers with 2 decimals  to help with visibility during coding

current_year = pd.Timestamp.today().year # define current year, so the workflow always creates a file per year

current_year # check current year

2025

In [21]:
# Create an output file, if there's none

# Defining file path and name
diretorio = r'C:\Users\guilh\OneDrive\Investimentos'
arquivo = os.path.join(diretorio, f"{current_year} - Operações na Bolsa - IR.xlsx")

# Creating directory, if it doesn't exist
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Creating file, if it doesn't exist
if not os.path.isfile(arquivo):

    # Creating a new workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Operações na Bolsa"

    # Saving the file
    wb.save(arquivo)

    # Declaring file creation
    print(f"Arquivo criado e salvo em: {arquivo}")

else:
    # Informting file existance
    print(f"Arquivo já existe")


Arquivo já existe


In [22]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('F2IRT - investment-registering-tool/AI Version/database/operations.db')

# Read the data from the database into a DataFrame
assets_df = pd.read_sql_query("SELECT * FROM operations", conn)

# Close the connection
conn.close()

assets_df.columns=['ID', 'Categoria', 'Operação', 'Ticker', 'Data da Operação', 'Preço unitário', 'Quantidade', 'Taxas - Liquidação', 'Taxas - Emolumentos', 'Taxas - Impostos', 'Taxas - Operacional', 'Taxas - Outros', 'Taxas - IRRF', 'Timestamp']

assets_df = assets_df.sort_values(by='Data da Operação', ascending=True).reset_index(drop=True)

# Display the first few rows of the DataFrame
assets_df.tail(10)

,ID,Categoria,Operação,Ticker,Data da Operação,Preço unitário,Quantidade,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Operacional,Taxas - Outros,Taxas - IRRF,Timestamp
103,102,Real Estate,Purchase,IRDM11,2024-12-23,59.74,9,0.14,0.03,0.00,0.00,0.00,0.00,2024-12-24 11:26:46.432000
104,103,Real Estate,Purchase,LVBI11,2024-12-23,94.89,5,0.13,0.02,0.00,0.00,0.00,0.00,2024-12-24 11:23:37.250000
105,116,Stock,Prize,ITUB4,2025-03-20,0.00,3,0.00,0.00,0.00,0.00,0.00,0.00,2025-06-26 08:11:23.488811
106,109,Stock,Purchase,BBAS3,2025-05-29,23.69,45,0.13,0.02,0.00,0.00,0.20,0.00,2025-05-30 11:26:12.052737
107,110,Real Estate,Purchase,XPML11,2025-05-29,104.40,1,0.09,0.02,0.00,0.00,0.00,0.00,2025-05-30 11:30:21.719293
108,111,Real Estate,Purchase,TRXF11,2025-05-29,102.00,1,0.09,0.02,0.00,0.00,0.00,0.00,2025-05-30 11:31:49.784918
109,115,Stock,Sale,NEOE3,2025-06-25,24.61,50,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:01:32.365289
110,112,Stock,Sale,ITUB4,2025-06-25,37.01,23,0.22,0.04,0.00,0.00,0.00,0.04,2025-06-26 07:58:51.020448
111,113,Stock,Sale,ITUB4,2025-06-25,37.02,12,0.11,0.02,0.00,0.00,0.00,0.02,2025-06-26 07:59:36.901600
112,114,Stock,Sale,ITSA4,2025-06-25,10.79,111,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:00:49.914304


# Retrieve value for each ticker

In [23]:
# Create df with tickers only

tickers = assets_df['Ticker'].drop_duplicates() # create a list with tickers for value searching

tickers.head(5) # show sample

0    BBSE3
1    BBDC4
2    BRAP4
3    BBAS3
4    GGBR4
Name: Ticker, dtype: object

In [24]:
# Search for each ticker's value
price_list = [] # create empty price list

for ticker in tickers: # for loop to search for each ticker's value and fill out list
    data = yf.Ticker(f'{ticker}.SA').info # create variable to connect with yfinance library to retrieve ticker's information
    price = data.get('currentPrice') # store current price in variable 'price'
    price_list.append(price) # add ticker's price to price list

price_list[:5] # show sample

[35.19, 16.55, 15.45, 21.6, 16.06]

In [25]:
# Create df with asset price for later use

asset_price  = pd.DataFrame({'Ticker': tickers, 'Valor Unitário Atual': price_list}) # create df with price for each ticker, combining ticker list with price list

asset_price.head(5) # show sample

,Ticker,Valor Unitário Atual
0,BBSE3,35.19
1,BBDC4,16.55
2,BRAP4,15.45
3,BBAS3,21.60
4,GGBR4,16.06


# Prepare Income Tax report

In [26]:
# Adjust quantity to negative when Sale, to consider actual current quantity for each asset

assets_df['Quantidade Transação'] = [(1 if row['Operação'] != 'Sale' else -1) * row['Quantidade'] # negative quantity if sale, to indicate reduction in portfolio
                                     for _, row in assets_df.iterrows()] # for every row in the df

assets_df[assets_df['Ticker'] == 'ITUB4']['Quantidade Transação'] # show sample


19     32
105     3
110   -23
111   -12
Name: Quantidade Transação, dtype: int64

In [27]:
# Determine purchase group, so the Preço Médio Atual and quantity renew every time a ticker is completely sold and later purchased again

# First, create a cumulative count to transactioned quantity, that will later be dropped

assets_df['Quantidade Atual'] = assets_df.groupby('Ticker')['Quantidade Transação'].cumsum() # accumulate transactioned quantity per ticker

# Now, sum 1 to Purchase Group for every first purchase after a complete ticker sale, to calculate Preço Médio Atual and quantity correctly

assets_df['Grupo de Compra'] = 1 # stablish 1 to all purchase groups at first

for row in range(len(assets_df)): # iterate with each row
    if assets_df.loc[row, 'Quantidade Atual'] == 0: # will execute whenever acc transactioned quantity is 0
        ticker = assets_df.loc[row, 'Ticker'] # store ticker for that case
        for next in range (row + 1, len(assets_df)): # iterate with all upcoming rows
            if assets_df.loc[next, 'Ticker'] == ticker: # will execute whenever it finds a new operation with that ticker
                assets_df.loc[next, 'Grupo de Compra'] += 1 # sum 1 to purchase group, indicating it's a new group to calculte Preço Médio Atual and quantity


print(assets_df.loc[assets_df['Ticker'] == 'ITUB4', ['Quantidade Atual', 'Grupo de Compra']]) # print specific example

     Quantidade Atual  Grupo de Compra
19                 32                1
105                35                1
110                12                1
111                 0                1


In [28]:
# Calculate buying price. Sales will be negative to represent revenue, and actual cost will naturally be less due to fees.

assets_df.loc[: , 'Preço Total'] = (assets_df['Preço unitário'] * assets_df['Quantidade Transação']) # Criar coluna Preço Total multiplicando Preço por Quantidade

# Calculate actual buying cost
assets_df.loc[: , 'Gasto Total'] = (assets_df['Preço Total'] # Calculate actual buying cost summing Preço Total with fees paid
+ assets_df['Taxas - Emolumentos']
+ assets_df['Taxas - IRRF']
+ assets_df['Taxas - Impostos']
+ assets_df['Taxas - Liquidação']
+ assets_df['Taxas - Operacional']
+ assets_df['Taxas - Outros']
)

assets_df.head(5) # show sample

,ID,Categoria,Operação,Ticker,Data da Operação,Preço unitário,Quantidade,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Operacional,Taxas - Outros,Taxas - IRRF,Timestamp,Quantidade Transação,Quantidade Atual,Grupo de Compra,Preço Total,Gasto Total
0,1,Stock,Purchase,BBSE3,2023-11-06,31.50,31,0.19,0.04,0.52,4.90,0.29,0.00,2023-12-30 12:39:37.608000,31,31,1,976.58,982.52
1,2,Stock,Purchase,BBDC4,2023-11-06,14.72,67,0.19,0.04,0.52,4.90,0.29,0.00,2023-12-30 12:41:33.760000,67,67,1,986.24,992.18
2,3,Stock,Purchase,BRAP4,2023-11-06,24.10,20,0.19,0.04,0.52,4.90,0.29,0.00,2023-12-30 12:44:19.580000,20,20,1,482.00,487.94
3,4,Stock,Purchase,BBAS3,2023-11-06,50.20,19,0.19,0.04,0.52,4.90,0.29,0.00,2023-12-30 12:46:25.333000,19,19,1,953.80,959.74
4,5,Stock,Purchase,GGBR4,2023-11-06,22.91,43,0.19,0.04,0.52,4.90,0.29,0.00,2023-12-30 12:46:25.333000,43,43,1,985.25,991.19


### Calculate average buy price after every transaction

This will be crucial for both calculating current buy price, as well as registering profit for each sale individually

In [29]:
# Calculate accumulated buy price without influence of Sales price, but accounting for the adjusted quantity after it with subsequent Preço Médio Atual rebalancing

# We'll create a for loop to iterate all rows in assets_df, that will take care of all necessary parameters and calculations

for i in range(len(assets_df)): # apply the following to each row in assets_df
    ticker = assets_df.loc[i, 'Ticker'] # create variable to store ticker for that row
    grupo_compra = assets_df.loc[i, 'Grupo de Compra'] # do the same for purchase group
    quant_trans = assets_df.loc[i, 'Quantidade Transação'] # do the same for transactioned quantity
    ttl_price = assets_df.loc[i, 'Preço Total'] # do the same for total price
    previous_rows = assets_df[(assets_df['Ticker'] == ticker) &
                              (assets_df['Grupo de Compra'] == grupo_compra) & 
                              (assets_df.index < i)] # create a sub-dataframe with all transactions in assets_df for the same ticker and purchase group, that happened before the current transaction

    # Since we'll need to pull data from previous rows, here we'll make sure not to break the code every first ticker and purchase group
    if previous_rows.empty: # apply if previous rows do not exist. that is: for every first transaction of each ticker and purchase group
        prev_cum_buy_price = 0 # stablish previous accumulated buy price as 0
        prev_cum_quant = 0 # stablish previous accumulated quantity as 0

    else: # if previous rows do exist, meaning all transactions after the first, now there is a baseline to compare
        prev_cum_buy_price = previous_rows['Preço Total Atual'].iloc[-1] # pull previous transactions' accumulated buy price
        prev_cum_quant = previous_rows['Quantidade Atual'].iloc[-1] # pull previous transactions' accumulated quantity
    
    # Now we'll calculate accumulated buy price, by pulling previous accumulated buy price and summing with current buy price. 
        # Bare in mind the process is fundamentally different between Sales and non-Sales transactions:
    if assets_df.loc[i, 'Operação'] != 'Sale': # apply if operation is not a Sale
        cum_buy_price = ttl_price + prev_cum_buy_price # accumulated buy price is total price plus previous row's accumulated buy price
    
    else: # apply if operation is Sale
        cum_buy_price = ((prev_cum_buy_price / prev_cum_quant) * quant_trans) + prev_cum_buy_price # accumulated buy price, but with a different approach:
        # pull previous row's average price and multiply by transactioned quantity to deduct from total accumulated buy price without changing avg price.
        # then sum resulted adjusted total price to previous accumulated buy price
    
    # Once we have the variable we want with the right values, all we need to do is bring it to the df
    assets_df.loc[i, 'Preço Total Atual'] = cum_buy_price # attribute values of variable with accumulated buy prices to column in assets_df


# Now that we have the accumulated buy price, we could just divide it by accumulated quantity to calculate average purchase price.
# However, that wouldn't work when the fully sell an asset, because this acc quantity goes to 0. So we'll go down a safer path:

# We'll iterate all rows from assets_df to make sure whenever we have a Sale, it will pull average price from previous row:
for i in range(len(assets_df)): # apply the following to each row in assets_df
    ticker = assets_df.loc[i, 'Ticker'] # create variable to store ticker for that row
    grupo_compra = assets_df.loc[i, 'Grupo de Compra'] # same for purchase group
    cum_buy_price = assets_df.loc[i, 'Preço Total Atual'] # same for accumulated buy price
    cum_quant = assets_df.loc[i, 'Quantidade Atual'] # same for accumulated quantity
    previous_rows = assets_df[(assets_df['Ticker'] == ticker) &
                              (assets_df['Grupo de Compra'] == grupo_compra) & 
                              (assets_df.index < i)] # create a sub-dataframe with all transactions in assets_df for the same ticker and purchase group, that happened before the current transaction

    # Since we'll need to pull data from previous rows, here we'll make sure not to break the code every first ticker and purchase group    
    if previous_rows.empty: # apply if previous rows do not exist. that is: for every first transaction of each ticker and purchase group
        prev_avg_price = 0 # stablish previous average price as 0
    
    else: # if previous rows do exist, meaning all transactions after the first, now there is a baseline to compare
        prev_avg_price = previous_rows['Preço Médio Atual'].iloc[-1] # pulls previous average price from previous row's average price column
    
    # As mentioned, the process is different for Sales. To avoid mistakes, we'll always pull average price from previous row's average price
    if assets_df.loc[i, 'Operação'] != 'Sale': # apply the following for non-Sales operations
        avg_price = cum_buy_price / cum_quant # average price is the division between accumulated buy price and accumulated quantity
    
    else: # apply the following for Sales operations
        avg_price = prev_avg_price # average price is the same as previous row's for the same ticker as purchase group

    # Once we have the variable we want with the right values, all we need to do is bring it to the df    
    assets_df.loc[i, 'Preço Médio Atual'] = avg_price    

assets_df.loc[assets_df['Ticker'] == 'ITUB4', ['Quantidade Atual','Preço Total Atual', 'Preço Médio Atual', 'Grupo de Compra']] # show specific example


,Quantidade Atual,Preço Total Atual,Preço Médio Atual,Grupo de Compra
19,32,982.72,30.71,1
105,35,982.72,28.08,1
110,12,336.93,28.08,1
111,0,0.00,28.08,1


In [30]:
# Export a detailed version of input file to the output file

file_path = fr"C:\Users\guilh\OneDrive\Investimentos\{current_year} - Operações na Bolsa - IR.xlsx" # define path to file. it will be used for all dfs

assets_df_sheet_name = 'Operações na Bolsa' # define sheet name

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    assets_df.to_excel(writer, sheet_name=assets_df_sheet_name, index=False) # execute instance to replace the sheet

print('Arquivo exportado com sucesso') # confirm file export

Arquivo exportado com sucesso


## Latest Portfolio Overview

File with most current overview of the portfolio, to summarize all information that will ultimately be added to your income tax report

In [31]:
assets_df[assets_df['Ticker'] == 'ITUB4']

last_idx = assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())
last_idx

C:\Users\guilh\AppData\Local\Temp\ipykernel_6960\226095869.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_idx = assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())


Ticker  Grupo de Compra
ALZR11  1                   89
BBAS3   1                  106
BBDC4   1                   23
BBSE3   1                   61
BEEF3   1                   29
BRAP4   1                    2
BTCI11  1                   33
BTLG11  1                   85
CMIG4   1                   75
CPTS11  1                   34
EGIE3   1                  101
ELET6   1                   81
GGBR4   1                   71
HGLG11  1                   18
HGRE11  1                   35
IRDM11  1                  103
ITSA4   1                  112
ITUB4   1                  111
KLBN11  1                   76
KLBN4   1                   77
LVBI11  1                  104
NEOE3   1                  109
PLCR11  1                   36
RAIZ4   1                   69
RBHG11  1                   30
RECR11  1                  100
ROMI3   1                   32
TAEE11  1                   24
TGAR11  1                   97
TRXF11  1                  108
UNIP6   1                   72
VALE3   1      

In [ ]:
# Create dataframe that brings only the latest transaction for each Ticker and Purchase Group, with most recent Quantity, Average Price and Accumulated Buy Price

latest_assets_df = assets_df.loc[assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())] # create the dataframe grouping by ticker and purchase group, bringing only the row where the max index, since you already sorted by date in the beginning

# Clean dataframe to keep only useful columns and assets that are still available in portfolio. Completely sold assets do not go here.
final_assets_df = latest_assets_df.loc[latest_assets_df['Quantidade Atual'] > 0 , # bring only assets that were not completely sold
                                       ['Ticker', 
                                        'Categoria', 
                                        'Grupo de Compra', 
                                        'Quantidade Atual', 
                                        'Preço Médio Atual', 
                                        'Preço Total Atual']] # define useful columns to bring

final_assets_df = pd.merge(final_assets_df, asset_price, on= 'Ticker', how='inner') # bring current market price for each asset

final_assets_df.head()  # show sample

C:\Users\guilh\AppData\Local\Temp\ipykernel_6960\1059231767.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  latest_assets_df = assets_df.loc[assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())] # create the dataframe grouping by ticker and purchase group, bringing only the row where the max index, since you already sorted by date in the beginning


,Ticker,Categoria,Grupo de Compra,Quantidade Atual,Preço Médio Atual,Preço Total Atual,Valor Unitário Atual
0,ALZR11,Real Estate,1,15,10.96,164.39,10.02
1,BBAS3,Stock,1,139,25.13,3493.09,21.60
2,BBSE3,Stock,1,87,32.15,2796.66,35.19
3,BRAP4,Stock,1,20,24.10,482.00,15.45
4,BTCI11,Real Estate,1,104,9.65,1004.12,9.02
5,BTLG11,Real Estate,1,13,99.65,1295.42,99.55
6,CMIG4,Stock,1,26,9.78,254.40,10.59
7,CPTS11,Real Estate,1,124,8.04,996.68,7.34
8,EGIE3,Stock,1,52,39.34,2045.53,43.20
9,ELET6,Stock,1,66,44.93,2965.33,43.99


In [33]:
# Calculate upside

# Calculate total current value of each asset
final_assets_df.loc[:, 'Valor Total Atual'] = final_assets_df['Valor Unitário Atual'] * final_assets_df['Quantidade Atual']

# Calculate financial upside by subtracting current total value and current total price
final_assets_df.loc[:, r'Upside (R$)'] = final_assets_df['Valor Total Atual'] - final_assets_df['Preço Total Atual']

# Calculate upside percentage by dividing current total value and current total price and subtracting 1
final_assets_df.loc[:, r'Upside %'] = final_assets_df['Valor Total Atual'] / final_assets_df['Preço Total Atual'] - 1

final_assets_df.head(5) # show sample


,Ticker,Categoria,Grupo de Compra,Quantidade Atual,Preço Médio Atual,Preço Total Atual,Valor Unitário Atual,Valor Total Atual,Upside (R$),Upside %
0,ALZR11,Real Estate,1,15,10.96,164.39,10.02,150.30,-14.09,-0.09
1,BBAS3,Stock,1,139,25.13,3493.09,21.60,3002.40,-490.69,-0.14
2,BBSE3,Stock,1,87,32.15,2796.66,35.19,3061.53,264.87,0.09
3,BRAP4,Stock,1,20,24.10,482.00,15.45,309.00,-173.00,-0.36
4,BTCI11,Real Estate,1,104,9.65,1004.12,9.02,938.08,-66.04,-0.07


In [34]:
# Drop assets already completely sold and export to excel

final_assets_df_sheet_name = 'Ativos em Patrimônio' # define sheet to register assets in portfolio

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    final_assets_df.to_excel(writer, sheet_name=final_assets_df_sheet_name, index=False) # execute instance to replace the sheet

print('O arquivo foi exportado com sucesso') # show sample

O arquivo foi exportado com sucesso


## Register sales that happened during the year

For each sale, bring average price on that occasion to calculate profit correctly and add it to the file

In [35]:
# Create df only with sales in current calendar year

sold_df = assets_df[(pd.to_datetime(assets_df['Data da Operação']).dt.year == current_year) & (assets_df['Operação'] == 'Sale')].drop(columns=[
    'Quantidade Transação', 
    'Quantidade Atual', 
    'Preço Total Atual']) # create df only with assets sold in current calendar year, without useless columns

sold_df.head(5) # show sample

,ID,Categoria,Operação,Ticker,Data da Operação,Preço unitário,Quantidade,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Operacional,Taxas - Outros,Taxas - IRRF,Timestamp,Grupo de Compra,Preço Total,Gasto Total,Preço Médio Atual
109,115,Stock,Sale,NEOE3,2025-06-25,24.61,50,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:01:32.365289,1,-1230.50,-1230.05,19.26
110,112,Stock,Sale,ITUB4,2025-06-25,37.01,23,0.22,0.04,0.00,0.00,0.00,0.04,2025-06-26 07:58:51.020448,1,-851.23,-850.93,28.08
111,113,Stock,Sale,ITUB4,2025-06-25,37.02,12,0.11,0.02,0.00,0.00,0.00,0.02,2025-06-26 07:59:36.901600,1,-444.24,-444.09,28.08
112,114,Stock,Sale,ITSA4,2025-06-25,10.79,111,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:00:49.914304,1,-1197.69,-1197.24,8.25


In [36]:
# Calculate gross and net profit

sold_df['Preço Total'] = abs(sold_df['Preço Total']) # convert Total Price to positive if negative. All sales have that column in negative.

sold_df['Gasto Total'] = abs(sold_df['Gasto Total']) # convert Total Spent to positive if negative.

# Calculate gross profit, by just subtracting total price and current total price, before fees
sold_df['Lucro Bruto'] = sold_df['Preço Total'] - (sold_df['Preço Médio Atual'] * sold_df['Quantidade'])

# Calculate net profit, by just subtracting total spent and current total price, after fees. This will be more useful for income purpuses.
sold_df['Lucro Líquido'] = sold_df['Gasto Total'] - (sold_df['Preço Médio Atual'] * sold_df['Quantidade'])

sold_df

,ID,Categoria,Operação,Ticker,Data da Operação,Preço unitário,Quantidade,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Operacional,Taxas - Outros,Taxas - IRRF,Timestamp,Grupo de Compra,Preço Total,Gasto Total,Preço Médio Atual,Lucro Bruto,Lucro Líquido
109,115,Stock,Sale,NEOE3,2025-06-25,24.61,50,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:01:32.365289,1,1230.50,1230.05,19.26,267.56,267.11
110,112,Stock,Sale,ITUB4,2025-06-25,37.01,23,0.22,0.04,0.00,0.00,0.00,0.04,2025-06-26 07:58:51.020448,1,851.23,850.93,28.08,205.44,205.14
111,113,Stock,Sale,ITUB4,2025-06-25,37.02,12,0.11,0.02,0.00,0.00,0.00,0.02,2025-06-26 07:59:36.901600,1,444.24,444.09,28.08,107.31,107.16
112,114,Stock,Sale,ITSA4,2025-06-25,10.79,111,0.33,0.06,0.00,0.00,0.00,0.06,2025-06-26 08:00:49.914304,1,1197.69,1197.24,8.25,281.44,280.99


In [37]:
# Export df to excel file

sold_df_sheet_name = 'Vendas do Ano' # define sheet name

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    sold_df.to_excel(writer, sheet_name=sold_df_sheet_name, index=False) # execute instance to replace the sheet

print('Arquivo exportado com sucesso') # confirm export

Arquivo exportado com sucesso
